In [1]:
import os
import sys 
import pandas as pd
from selenium import webdriver 
import win32com.client
import time
from datetime import date
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

import concurrent.futures

import requests

import threading
import shutil


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [2]:
#Open ID's Excel

top_talents = pd.read_excel(r'C:\Users\bd1266\##\Photos Automation\TopTalents.xlsx')


In [3]:
final_directory = "C:\\Users\\##\\Images_"

In [4]:
top_talents['Employee ID'] = top_talents['Employee ID'].astype(str)


In [5]:
top_talents['Employee ID'] = top_talents['Employee ID'].apply(lambda x: str(x).zfill(6))

In [6]:
top_talents['Employee ID'][7]

'003105'

In [17]:
def employee_photo(empid, empname):
    
    print(f"Starting process for {empname}")
    file_name = str(empid) + " - " + empname + ".jpg"  # Set the desired file name
    download_directory = "C:\\Users\\bd1266\\Downloads\\Photo Downloads"
    # Set Chrome options
    options = webdriver.ChromeOptions()
    prefs = {
            "download.default_directory": download_directory,
             "download.prompt_for_download": False,
             "download.directory_upgrade": True,
             "safebrowsing.enabled": True,
             "safebrowsing.disable_download_protection": True}
    options.add_experimental_option("prefs", prefs)
    


    driver = webdriver.Chrome(options=options)

   

    driver.get("https://www.myworkday.com/##/d/home.htmld")

    time.sleep(25)



    
   
    for i in range(3):
        try:
            search_input =  WebDriverWait(driver, 25).until(
                    EC.presence_of_element_located((
                        By.XPATH, "//input[contains(@data-automation-id, 'globalSearchInput')]"
                    ))
            )
            time.sleep(5)
            search_input.send_keys(empid)
            time.sleep(10)
            search_input.send_keys(Keys.ENTER)
            time.sleep(15)
            time.sleep(5)
        except:
            time.sleep(10)
        else:
            break
    assert search_input
    
    
    try:
        # enter the profile page
        link_element = WebDriverWait(driver, 25).until(
            EC.presence_of_element_located((
                By.XPATH, "//a[contains(@data-automation-id, 'pex-search-results-header-title-link')]"
            ))
        )
        link_element.click()
        time.sleep(15)
    except Exception as e:
        print(f"could not enter to the profile page: {e}")
        driver.quit()
        return
        # get the small profile picture element
    try:

        img_element_= WebDriverWait(driver, 25).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "img[alt*='employee photo']"))
        )
    except Exception as e:
        print(f"""This employee: {empname} seems to have no photo. 
              Error: {e}""")
        driver.quit()
        return

    img_element_.click()
        #img_element.click()

    time.sleep(10)

            
        # find the element we are actually looking for
    try:
        img_element_1 = WebDriverWait(driver, 25).until(
            EC.presence_of_element_located((By.CLASS_NAME, "WO0P"))
        )
        try:
            #if the image was found, lets try to download it with the find_elements method. Data is more accesible that way.
            img_element_1_link = driver.find_elements(By.CLASS_NAME, "WO0P")[0].get_attribute("src")
            # download picture
            driver.get(img_element_1_link)
            driver.quit()
        except:
            time.sleep(5)
            img_element_1_link = driver.find_elements(By.CLASS_NAME, "WO0P")[0].get_attribute("src")
            # download picture
            driver.get(img_element_1_link)
            driver.quit()
            
    except Exception as e:
        print(f"failed for employee {empname} id {empid} error {e}")
        driver.quit()
        return
    
           #move file
    print("file downloaded")
    files = os.listdir(download_directory)
    assert len(files) == 1, "Expected exactly one file in the download directory."
    shutil.move(os.path.join(download_directory, files[0]),
        os.path.join(final_directory, file_name))


In [ ]:
for i in range(len(top_talents)):
    
    empid = top_talents["Employee ID"][i]
    empname = top_talents["Full Name"][i]
    employee_photo(empid, empname)